# Objetivo Geral:

**Construir um algoritmo que classifique os comentários calculando a probabilidade de se recomendar o produto para algum(a) amigo(a)**

### Objetivo específico desse notebook:

- Preparar a base de treino, teste e validação.

# 0.0 Imports

In [2]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation
import pandas as pd
import re

# Config path and dir
import pathlib
import os


## 0.1 Data Loading

### 0.1.1 Caminhos de diretórios e arquivos

In [3]:
#os.path.abspath('..')

In [4]:
BASE_DIR = pathlib.Path().resolve().parent.absolute()
DATA_DIR = os.path.join(BASE_DIR, 'data')
DATA_RAW_DIR = os.path.join(DATA_DIR, 'raw')
DATA_INTERIM_DIR = os.path.join(DATA_DIR, 'interim')
DATA_RAW_FILE = os.path.join(DATA_RAW_DIR, 'review.csv')

### 0.1.2 Load

In [5]:
path = rf'{DATA_RAW_FILE}'
df = pd.read_csv(path)

C:\Users\kayss\AppData\Local\Temp\ipykernel_7156\3283744367.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


# 1.0 Data Description

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   submission_date        132373 non-null  object 
 1   reviewer_id            132373 non-null  object 
 2   product_id             132373 non-null  object 
 3   product_name           132289 non-null  object 
 4   product_brand          40982 non-null   object 
 5   site_category_lv1      132367 non-null  object 
 6   site_category_lv2      128360 non-null  object 
 7   review_title           132071 non-null  object 
 8   overall_rating         132373 non-null  int64  
 9   recommend_to_a_friend  132355 non-null  object 
 10  review_text            129098 non-null  object 
 11  reviewer_birth_year    126389 non-null  float64
 12  reviewer_gender        128237 non-null  object 
 13  reviewer_state         128382 non-null  object 
dtypes: float64(1), int64(1), object(12)


In [8]:
df.tail()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
132368,2018-05-31 23:30:50,15f20e95ff44163f3175aaf67a5ae4a94d5030b409e521...,17962233,Carregador De Pilha Sony + 4 Pilhas Aa 2500mah,NaN,Câmeras e Filmadoras,Acessórios para Câmeras e Filmadoras,Ótimo produto!,5,Yes,"Vale muito, estou usando no controle do Xbox e...",1988.0,M,RS
132369,2018-05-31 23:42:25,def7cf9028b0673ab8bca3b1d06e085461fafb88cd48d9...,132631701,Mop Giratório Fit + Refil Extra - At Home,NaN,Utilidades Domésticas,Material de Limpeza,Sensacional,5,Yes,"Prático e barato, super indico o produto para ...",1979.0,F,SP
132370,2018-05-31 23:44:16,7bcbf542f5d7dd9a9a192a6805adba7a7a4c1ce3bf00df...,16095859,Fita Led 5m Rgb 3528 Siliconada Com 300 Leds C...,NaN,Automotivo,Iluminação,Ótimo produto,4,Yes,Chegou antes do prazo previsto e corresponde a...,1979.0,F,PR
132371,2018-05-31 23:46:48,e6fb0b19277d01c2a300c7837a105f3c369377e92f9c19...,6774907,Etiquetas Jurídicas Vade Mecum - Marca Fácil,marca facil,Papelaria,Material de Escritório,O produto não é bom.,1,No,"Material fraco, poderia ser melhor. Ficou deve...",1991.0,M,RJ
132372,2018-05-31 23:50:33,ea9467aa73271fb4f68c04f4dd4f4eef304d6ee85441fb...,114081902,Painel de Fotos Bee Colection Rue Bac (74x94x3...,kapos,Decoração,Painel de Fotos,Produto não entregue,1,No,"Comprei esse produto, quando chegou estava com...",1982.0,F,ES


**OBSERVAÇÕES:** 
- 14 Colunas: 2 Datas, 11 categóricas e 1 Númerica.
- Há dados faltantes em algumas colunas, que dentre elas está o Review_text

# 2.0 Data Preparation

## 2.1 Data Filtering

In [10]:
df_filtrado = df.loc[~df['review_text'].isnull()] # Filtrando linhas que tem valor null no atributo de comentários.
df_filtrado = df_filtrado.loc[~df_filtrado['recommend_to_a_friend'].isnull()] # Filtrando linhas que tem valor null no atributo de recomendação.

In [11]:
# Confirmando se há o mesmo número de linhas para a recomendação e revisão de texto
df_filtrado[['recommend_to_a_friend','review_text']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129080 entries, 0 to 132372
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   recommend_to_a_friend  129080 non-null  object
 1   review_text            129080 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


In [12]:
df_filtrado_2 = df_filtrado[['recommend_to_a_friend','review_text']]

## 2.2 Aplicando funções de tratamento

In [14]:
# Remove os Emojis
def _remove_emoji(string):
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [15]:
def _sub_char_especial_normal(text):
    # Substitui caracteres especiais por letras com acentos

    # Letra minúsculas
    text = re.sub(r"&aacute;", "á", text)
    text = re.sub(r"&atilde;", "ã", text)
    text = re.sub(r"&agrave;", "à", text)
    text = re.sub(r"&acirc;", "â", text)
    text = re.sub(r"&aring;", "a", text)
    text = re.sub(r"&auml;", "a", text)
    text = re.sub(r"&aelig;", "ae", text)
    text = re.sub(r"&eacute;", "é", text)
    text = re.sub(r"&ecirc;", "ê", text)
    text = re.sub(r"&egrave;", "è", text)
    text = re.sub(r"&euml;", "e", text)
    text = re.sub(r"&iacute;", "í", text)
    text = re.sub(r"&icirc;", "î", text)
    text = re.sub(r"&iuml;", "i", text)
    text = re.sub(r"&oacute;", "ó", text)
    text = re.sub(r"&otilde;", "õ", text)
    text = re.sub(r"&ocirc;", "ô", text)
    text = re.sub(r"&ograve;", "ò", text)
    text = re.sub(r"&uacute;", "ú", text)
    text = re.sub(r"&ucirc;", "û", text)
    text = re.sub(r"&ugrave;", "ù", text)
    text = re.sub(r"&uuml;", "u", text)
    text = re.sub(r"&ccedil;", "ç", text)
    text = re.sub(r"&ntilde;", "não", text)

    # Letras maiúsculas
    text = re.sub(r"&Aacute;", "Á", text)
    text = re.sub(r"&Atilde;","Ã", text) # Letra maiúscula
    text = re.sub(r"&Agrave;", "À", text)
    text = re.sub(r"&Acirc;", "Â", text)
    text = re.sub(r"&Aring;", "A", text)
    text = re.sub(r"&Auml;", "A", text)
    text = re.sub(r"&Aelig;", "AE", text)
    text = re.sub(r"&Eacute;", "É", text)
    text = re.sub(r"&Ecirc;", "Ê", text)
    text = re.sub(r"&Egrave;", "È", text)
    text = re.sub(r"&Euml;", "E", text)
    text = re.sub(r"&Iacute;", "Í", text)
    text = re.sub(r"&Icirc;", "Î", text)
    text = re.sub(r"&Iuml;", "I", text)
    text = re.sub(r"&Oacute;", "Ó", text)
    text = re.sub(r"&Otilde;", "Õ", text)
    text = re.sub(r"&Ocirc;", "Ô", text)
    text = re.sub(r"&Ograve;", "Ò", text)
    text = re.sub(r"&Uacute;", "Ú", text)
    text = re.sub(r"&Ucirc;", "Û", text)
    text = re.sub(r"&Ugrave;", "Ù", text)
    text = re.sub(r"&Uuml;", "U", text)
    text = re.sub(r"&Ccedil;", "Ç", text)
    text = re.sub(r"&Ntilde;", "Não", text)
    text = re.sub(r"&Yacute;", "Y", text)

    #any special char to be replaced
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"&amp;", "&", text)
    text = re.sub(r"&reg;", "&", text) 
    text = re.sub(r"&copy;", "&", text) 
    text = re.sub(r"&quot;","'", text)
    text = re.sub(r"&lsquo;","'", text)
    text = re.sub(r"&rsquo;","'", text)
    return text

In [22]:
# Termos em code HTML
def _remove_char_html(text):
    text = re.sub(r"&ndash;","", text)
    text = re.sub(r"&ordf;","", text)
    text = re.sub(r"&nbsp;","", text)
    text = re.sub(r"&cent;","", text)
    text = re.sub(r"&pound;","", text)
    text = re.sub(r"&sect;","", text)
    text = re.sub(r"&copy;","", text)
    text = re.sub(r"&laquo;","", text)
    text = re.sub(r"&raquo;","", text)
    text = re.sub(r"&reg;","", text)
    text = re.sub(r"&deg;","", text)
    text = re.sub(r"&plusmn;","", text)
    text = re.sub(r"&para;","", text)
    text = re.sub(r"&middot;","", text)
    text = re.sub(r"&frac12;","", text)
    text = re.sub(r"&mdash;","", text)
    text = re.sub(r"&lsquo;","", text)
    text = re.sub(r"&rsquo;","", text)
    text = re.sub(r"&sbquo;","", text)
    text = re.sub(r"&ldquo;","", text)
    text = re.sub(r"&rdquo;","", text)
    text = re.sub(r"&bdquo;","", text)
    text = re.sub(r"&dagger;","", text)
    text = re.sub(r"&Dagger;","", text)
    text = re.sub(r"&bull;","", text)
    text = re.sub(r"&hellip;","", text)
    text = re.sub(r"&prime;","", text)
    text = re.sub(r"&Prime;","", text)
    text = re.sub(r"&euro;","", text)
    text = re.sub(r"&trade;","", text)
    text = re.sub(r"&asymp;","", text)
    text = re.sub(r"&ne;","", text)
    text = re.sub(r"&le;","", text)
    text = re.sub(r"&ge;","", text)
    text = re.sub(r"&lt;","", text)
    text = re.sub(r"&gt;","", text)
    return text

In [23]:
# Remove as tags de HTML
def _remove_tags_html(text):
    text = re.sub('<[^<]+?>', '', str(text))
    return text


In [24]:
# Remove web sites:
def _remove_web(text):
    text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?]))",
                "xxxWEBxxx",
                str(text)
            )
    text = ' '.join([w for w in text.split() if '.com' not in w])
    return text

In [25]:
# Remove números
def _remove_numbers(text):
    text = re.sub(r'[0-9]+', '', text)
    return text

def _remove_pontuação(text):
    text = re.sub(r'[^\w\s]','',text)
    return text

In [26]:
def transform_text(text):
    text = _remove_tags_html(text)
    text = _remove_web(text)
    text = _remove_emoji(text)
    text = _sub_char_especial_normal(text)
    text = _remove_char_html(text)
    text = _remove_numbers(text)
    text = _remove_pontuação(text)
    return text

In [27]:
teste = str(df_filtrado_2['review_text'].sample().tolist())
teste

"['Produto de qualidade, de fácil reposição do fio de nylon, durável. Recomendo!']"

In [28]:
transform_text(teste)

'Produto de qualidade de fácil reposição do fio de nylon durável Recomendo'

In [29]:
df_filtrado_2['review_text'] = df_filtrado_2['review_text'].apply(str)
df_filtrado_2['review_text'] = df_filtrado_2['review_text'].apply(transform_text)

C:\Users\kayss\AppData\Local\Temp\ipykernel_7156\1040504740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_2['review_text'] = df_filtrado_2['review_text'].apply(str)
C:\Users\kayss\AppData\Local\Temp\ipykernel_7156\1040504740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_2['review_text'] = df_filtrado_2['review_text'].apply(transform_text)


## 2.3 Aplicando dicionário na label

In [30]:
df_filtrado_2['recommend_to_a_friend'].value_counts()

Yes    95066
No     34014
Name: recommend_to_a_friend, dtype: int64

In [31]:
dicionario = {
    'Yes': 1,
    'No' : 0
}

df_filtrado_2['recommend_to_a_friend_num'] = df_filtrado_2['recommend_to_a_friend'].map(dicionario)

C:\Users\kayss\AppData\Local\Temp\ipykernel_7156\45278559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_2['recommend_to_a_friend_num'] = df_filtrado_2['recommend_to_a_friend'].map(dicionario)


In [32]:
df_filtrado_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129080 entries, 0 to 132372
Data columns (total 3 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   recommend_to_a_friend      129080 non-null  object
 1   review_text                129080 non-null  object
 2   recommend_to_a_friend_num  129080 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.9+ MB


## 2.4 Train and test

In [33]:
df_random = df_filtrado_2.sample(129080,random_state=42)
df_train = df_random[:100000]
df_test = df_random[100000:]
print(df_train['recommend_to_a_friend_num'].value_counts(normalize=True))
print(df_test['recommend_to_a_friend_num'].value_counts(normalize=True))

1    0.73604
0    0.26396
Name: recommend_to_a_friend_num, dtype: float64
1    0.738033
0    0.261967
Name: recommend_to_a_friend_num, dtype: float64


In [34]:
path_save_test = os.path.join(DATA_INTERIM_DIR, 'validation_v1.csv')
df_test.to_csv(path_save_test)

In [35]:
path_save_train = os.path.join(DATA_INTERIM_DIR, 'train_v1.csv')
df_train.to_csv(path_save_train)